In [19]:
import numpy as np
#from lsst.sims.utils import m5_flat_sed

In [20]:
def m5_flat_sed(visitFilter, musky, FWHMeff, expTime, airmass, tauCloud=0):
    """Calculate the m5 value, using photometric scaling.  Note, does not include shape of the object SED.

    Parameters
    ----------
    visitFilter : str
         One of u,g,r,i,z,y
    musky : float
        Surface brightness of the sky in mag/sq arcsec
    FWHMeff : float
        The seeing effective FWHM (arcsec)
    expTime : float
        Exposure time for the entire visit in seconds
    airmass : float
        Airmass of the observation (unitless)
    tauCloud : float (0.)
        Any extinction from clouds in magnitudes (positive values = more extinction)

    Output
    ------
    m5 : float
        The five-sigma limiting depth of a point source observed in the given conditions.
    """

    # Set up expected extinction (kAtm) and m5 normalization values (Cm) for each filter.
    # The Cm values must be changed when telescope and site parameters are updated.
    #
    # These values are calculated using $SYSENG_THROUGHPUTS/python/calcM5.py.
    # This set of values are calculated using v1.2 of the SYSENG_THROUGHPUTS repo.

    # Only define the dicts once on initial call
    if not hasattr(m5_flat_sed, 'Cm'):
        m5_flat_sed.Cm = {'u': 23.067,
                          'g': 24.414,
                          'r': 24.439,
                          'i': 24.325,
                          'z': 24.157,
                          'y': 23.730}
        m5_flat_sed.dCm_infinity = {'u': 0.631,
                                    'g': 0.179,
                                    'r': 0.097,
                                    'i': 0.071,
                                    'z': 0.048,
                                    'y': 0.037}
        m5_flat_sed.kAtm = {'u': 0.492,
                            'g': 0.213,
                            'r': 0.126,
                            'i': 0.096,
                            'z': 0.069,
                            'y': 0.170}
        m5_flat_sed.msky = {'u': 22.989,
                            'g': 22.256,
                            'r': 21.196,
                            'i': 20.478,
                            'z': 19.600,
                            'y': 18.612}
    # Calculate adjustment if readnoise is significant for exposure time
    # (see overview paper, equation 7)
    Tscale = expTime / 30.0 * np.power(10.0, -0.4 * (musky - m5_flat_sed.msky[visitFilter]))
    dCm = 0.
    dCm += m5_flat_sed.dCm_infinity[visitFilter]
    dCm -= 1.25 * np.log10(1 + (10**(0.8 * m5_flat_sed.dCm_infinity[visitFilter]) - 1)/Tscale)
    print('dCm', dCm)
    # Calculate fiducial m5
    m5 = (m5_flat_sed.Cm[visitFilter] + dCm + 0.50 * (musky - 21.0) + 2.5 * np.log10(0.7 / FWHMeff) +
          1.25 * np.log10(expTime / 30.0) - m5_flat_sed.kAtm[visitFilter] * (airmass - 1.0) - 1.1 * tauCloud)

    return m5


In [29]:
m1 = m5_flat_sed('z', 20., 1., 30., 1.2)
m2 = m5_flat_sed('z', 20., 1., 15., 1.2)

dCm -0.0200868653569
dCm -0.0805780479209


In [30]:
m1,m2

(23.235858234678751, 22.799079557534736)

In [31]:
def coadd_m5(m5s):
    result = 1.25*np.log10(np.sum(10.**(0.8*m5s)))
    return result

In [32]:
coadd_m5(np.array([m2,m2]))

23.175367052114716

In [ ]:
# YIKES! That does not seem to be behaving properly at all, 2x15s is not adding up to 1x30s.
# Ah, this dCm stuff is clearly an issue 